In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
torch.__version__

'1.0.1'

In [3]:
BATCH_SIZE = 50
EPOCHS = 1

In [4]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=False, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [28]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(), #(0,1)
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [35]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,16,5,1,2) # 10, 24x24
        self.conv2=nn.Conv2d(16,32,5,1,2) # 128, 10x10
        self.fc1 = nn.Linear(32*7*7,10)
        #self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        
        out = F.log_softmax(out,dim=1)
        return out

In [36]:
model = ConvNet()
optimizer = optim.Adam(model.parameters())
print(model)

ConvNet(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
)


In [37]:
params = list(model.parameters())
print(params)

[Parameter containing:
tensor([[[[-0.0684, -0.1642, -0.0700,  0.0043,  0.1737],
          [-0.1744, -0.0160, -0.1015, -0.1403, -0.1834],
          [ 0.1236,  0.1964,  0.0375,  0.0235,  0.0386],
          [-0.1053, -0.1916, -0.1605, -0.1984, -0.1818],
          [-0.0016,  0.0487,  0.0192,  0.1956, -0.1426]]],


        [[[-0.0845, -0.1468, -0.0873,  0.0452,  0.0057],
          [-0.1679,  0.1916,  0.0763,  0.0342, -0.0944],
          [-0.0533, -0.1918,  0.1823,  0.0154, -0.0661],
          [-0.1699,  0.1125,  0.1694, -0.0302,  0.0068],
          [-0.0743,  0.1297, -0.0584,  0.1711, -0.1236]]],


        [[[ 0.1470, -0.0086, -0.1463,  0.1658,  0.1037],
          [ 0.0434, -0.1511,  0.0050,  0.0580, -0.1216],
          [-0.1829, -0.1616, -0.1467,  0.1850,  0.0640],
          [-0.0552, -0.1475, -0.1072, -0.0674,  0.1750],
          [ 0.0113,  0.1916,  0.1972,  0.1973, -0.1736]]],


        [[[ 0.0376, -0.0238, -0.0141, -0.1811, -0.1811],
          [ 0.0206, -0.0119,  0.0295,  0.0870, -0.111

In [38]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [39]:
def test(model,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [40]:
for epoch in range(1, EPOCHS + 1):
    train(model,train_loader, optimizer, epoch)
    test(model,test_loader)

RuntimeError: size mismatch, m1: [50 x 6272], m2: [1568 x 10] at c:\a\w\1\s\tmp_conda_3.6_062715\conda\conda-bld\pytorch-cpu_1550384979956\work\aten\src\th\generic/THTensorMath.cpp:940